# Data Mining Assignment 2

Group 20
Ziyi Shan
Yuyuan Zhao
Fangshu Qu
Yidan Wang

## Imports and inits

In [2]:
import h2o
from h2o.estimators.glm import H2OGeneralizedLinearEstimator
from h2o.estimators.glrm import H2OGeneralizedLowRankEstimator
from h2o.grid.grid_search import H2OGridSearch 
h2o.init(max_mem_size='12G') # give h2o as much memory as possible
h2o.no_progress() # turn off h2o progress bars

import numpy as np
import pandas as pd

import matplotlib as plt
%matplotlib inline

Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O cluster uptime:,8 days 0 hours 9 mins
H2O cluster timezone:,America/New_York
H2O data parsing timezone:,UTC
H2O cluster version:,3.22.1.3
H2O cluster version age:,1 month and 3 days
H2O cluster name:,H2O_from_python_suzy_yaagn4
H2O cluster total nodes:,1
H2O cluster free memory:,1.853 Gb
H2O cluster total cores:,8
H2O cluster allowed cores:,8
H2O cluster status:,"locked, healthy"


import the datasets

In [2]:
train = pd.read_csv('train.csv')
test  = pd.read_csv('test.csv')
print(train.shape)
print(test.shape)

(160057, 14)
(151484, 13)


In [3]:
train.head(5)

,id,chain,offer,market,repeater,offerdate,category,quantity,company,offervalue,brand,Average_Purchasequantity,Average_Purchaseamount,FACTOR
0,86246,205,1208251,34,t,2013-04-24,2202,1,104460040,2.00,3718,0.0,0.000,0
1,86252,205,1197502,34,t,2013-03-27,3203,1,106414464,0.75,13474,1.0,2.425,1
2,12682470,18,1197502,11,f,2013-03-28,3203,1,106414464,0.75,13474,1.0,2.500,1
3,12996040,15,1197502,9,f,2013-03-25,3203,1,106414464,0.75,13474,0.0,0.000,0
4,13089312,15,1204821,9,f,2013-04-01,5619,1,107717272,1.50,102504,0.0,0.000,0


In [4]:
test.head(5)

,id,chain,offer,market,offerdate,category,quantity,company,offervalue,brand,Average_Purchaseamount,AVG_purchasequantity,Factor
0,12262064,95,1219903,39,2013-06-27,799,1,1076211171,1.5,17286,0.000000,0.0,0
1,12277270,95,1221658,39,2013-06-23,7205,2,103700030,3.0,4294,3.882000,1.2,1
2,12332190,95,1213242,39,2013-06-15,5824,1,105190050,2.0,26456,6.980000,2.0,1
3,12524696,4,1221665,1,2013-06-20,7205,1,103700030,1.5,4294,4.242500,1.0,1
4,13074629,14,1221658,8,2013-06-21,7205,2,103700030,3.0,4294,5.618571,1.0,1


Recode the target variable, repeater, such that t=1 and f=0.

In [5]:
train['repeater'].replace(to_replace='t', value=1, inplace=True)
train['repeater'].replace(to_replace='f', value=0, inplace=True)
train.head(5)

,id,chain,offer,market,repeater,offerdate,category,quantity,company,offervalue,brand,Average_Purchasequantity,Average_Purchaseamount,FACTOR
0,86246,205,1208251,34,1,2013-04-24,2202,1,104460040,2.00,3718,0.0,0.000,0
1,86252,205,1197502,34,1,2013-03-27,3203,1,106414464,0.75,13474,1.0,2.425,1
2,12682470,18,1197502,11,0,2013-03-28,3203,1,106414464,0.75,13474,1.0,2.500,1
3,12996040,15,1197502,9,0,2013-03-25,3203,1,106414464,0.75,13474,0.0,0.000,0
4,13089312,15,1204821,9,0,2013-04-01,5619,1,107717272,1.50,102504,0.0,0.000,0


## category

In [6]:
a1=np.setdiff1d(train['category'],test['category'])
print(a1)

[1703 3203 3509 4401 9909]


In [7]:
b1=np.setdiff1d(test['category'],train['category'])
print(b1)

[ 706  799 4517 5122 5824 7205 9115]


In [8]:
train['category'].replace(to_replace = a1,
                          value="unknown", inplace = True)
print(train)

                id  chain    offer  market  repeater   offerdate category  \
0            86246    205  1208251      34         1  2013-04-24     2202   
1            86252    205  1197502      34         1  2013-03-27  unknown   
2         12682470     18  1197502      11         0  2013-03-28  unknown   
3         12996040     15  1197502       9         0  2013-03-25  unknown   
4         13089312     15  1204821       9         0  2013-04-01     5619   
5         13179265     14  1197502       8         0  2013-03-29  unknown   
6         13251776     15  1200581       9         0  2013-03-30     1726   
7         13540129     14  1200581       8         0  2013-03-30     1726   
8         13807224      4  1204576       1         0  2013-04-05     5616   
9         13873775      4  1197502       1         0  2013-03-26  unknown   
10        13974451      4  1197502       1         0  2013-03-26  unknown   
11        14088807      3  1200581       2         1  2013-04-17     1726   

[160057 rows x 14 columns]


In [9]:
test['category'].replace(to_replace = b1,
                          value="unknown", inplace = True)
print(test)

                id  chain    offer  market   offerdate category  quantity  \
0         12262064     95  1219903      39  2013-06-27  unknown         1   
1         12277270     95  1221658      39  2013-06-23  unknown         2   
2         12332190     95  1213242      39  2013-06-15  unknown         1   
3         12524696      4  1221665       1  2013-06-20  unknown         1   
4         13074629     14  1221658       8  2013-06-21  unknown         2   
5         13387341     14  1221658       8  2013-06-22  unknown         2   
6         13501141      4  1203439       1  2013-05-13  unknown         1   
7         13558712     15  1203439       9  2013-05-12  unknown         1   
8         13563017     14  1221665       8  2013-06-24  unknown         1   
9         13584134     14  1221665       8  2013-06-27  unknown         1   
10        13744500      4  1221658       1  2013-06-26  unknown         2   
11        14026320     14  1221663       8  2013-06-23  unknown         1   

[151484 rows x 13 columns]


## brand

In [10]:
a2 = np.setdiff1d(train['brand'],test['brand'])
print(a2)

[  875  6732 13474 13791 28840]


In [12]:
b2 = np.setdiff1d(test['brand'],train['brand'])
print(b2)

[ 1322  4294 17286 17311 26189 26456 93904]


In [13]:
train['brand'].replace(to_replace = a2,value="unknown", inplace = True)
print(train)

                id  chain    offer  market  repeater   offerdate category  \
0            86246    205  1208251      34         1  2013-04-24     2202   
1            86252    205  1197502      34         1  2013-03-27  unknown   
2         12682470     18  1197502      11         0  2013-03-28  unknown   
3         12996040     15  1197502       9         0  2013-03-25  unknown   
4         13089312     15  1204821       9         0  2013-04-01     5619   
5         13179265     14  1197502       8         0  2013-03-29  unknown   
6         13251776     15  1200581       9         0  2013-03-30     1726   
7         13540129     14  1200581       8         0  2013-03-30     1726   
8         13807224      4  1204576       1         0  2013-04-05     5616   
9         13873775      4  1197502       1         0  2013-03-26  unknown   
10        13974451      4  1197502       1         0  2013-03-26  unknown   
11        14088807      3  1200581       2         1  2013-04-17     1726   

[160057 rows x 14 columns]


In [14]:
test['brand'].replace(to_replace = b2,value="unknown", inplace = True)
print(test)

                id  chain    offer  market   offerdate category  quantity  \
0         12262064     95  1219903      39  2013-06-27  unknown         1   
1         12277270     95  1221658      39  2013-06-23  unknown         2   
2         12332190     95  1213242      39  2013-06-15  unknown         1   
3         12524696      4  1221665       1  2013-06-20  unknown         1   
4         13074629     14  1221658       8  2013-06-21  unknown         2   
5         13387341     14  1221658       8  2013-06-22  unknown         2   
6         13501141      4  1203439       1  2013-05-13  unknown         1   
7         13558712     15  1203439       9  2013-05-12  unknown         1   
8         13563017     14  1221665       8  2013-06-24  unknown         1   
9         13584134     14  1221665       8  2013-06-27  unknown         1   
10        13744500      4  1221658       1  2013-06-26  unknown         2   
11        14026320     14  1221663       8  2013-06-23  unknown         1   

[151484 rows x 13 columns]


## brand

In [15]:
a3 = np.setdiff1d(train['offer'],test['offer'])
print(a3)

[1194044 1197502 1199256 1199258 1200578 1200579 1200988 1203052]


In [16]:
b3 = np.setdiff1d(test['offer'],train['offer'])
print(b3)

[1190530 1203439 1213242 1219900 1219903 1220502 1220503 1221658 1221663
 1221665 1221666 1221667 1230218]


In [17]:
train['offer'].replace(to_replace = a3,value="unknown", inplace = True)
print(train)

                id  chain    offer  market  repeater   offerdate category  \
0            86246    205  1208251      34         1  2013-04-24     2202   
1            86252    205  unknown      34         1  2013-03-27  unknown   
2         12682470     18  unknown      11         0  2013-03-28  unknown   
3         12996040     15  unknown       9         0  2013-03-25  unknown   
4         13089312     15  1204821       9         0  2013-04-01     5619   
5         13179265     14  unknown       8         0  2013-03-29  unknown   
6         13251776     15  1200581       9         0  2013-03-30     1726   
7         13540129     14  1200581       8         0  2013-03-30     1726   
8         13807224      4  1204576       1         0  2013-04-05     5616   
9         13873775      4  unknown       1         0  2013-03-26  unknown   
10        13974451      4  unknown       1         0  2013-03-26  unknown   
11        14088807      3  1200581       2         1  2013-04-17     1726   

[160057 rows x 14 columns]


In [18]:
test['offer'].replace(to_replace = b3,value="unknown", inplace = True)
print(test)

                id  chain    offer  market   offerdate category  quantity  \
0         12262064     95  unknown      39  2013-06-27  unknown         1   
1         12277270     95  unknown      39  2013-06-23  unknown         2   
2         12332190     95  unknown      39  2013-06-15  unknown         1   
3         12524696      4  unknown       1  2013-06-20  unknown         1   
4         13074629     14  unknown       8  2013-06-21  unknown         2   
5         13387341     14  unknown       8  2013-06-22  unknown         2   
6         13501141      4  unknown       1  2013-05-13  unknown         1   
7         13558712     15  unknown       9  2013-05-12  unknown         1   
8         13563017     14  unknown       8  2013-06-24  unknown         1   
9         13584134     14  unknown       8  2013-06-27  unknown         1   
10        13744500      4  unknown       1  2013-06-26  unknown         2   
11        14026320     14  unknown       8  2013-06-23  unknown         1   

[151484 rows x 13 columns]


# Part2 

In [3]:
train = h2o.import_file('train.csv')
test = h2o.import_file('test.csv')